In [1]:
%cd ../search/search
#! python pacman.py --layout originalClassic

/home/gabriel/Documentos/Tarefas/Universidade/MC886/projects_MachineLearning/project3_ra214129_ra216179/search/search


In [2]:
from pacman import runGames, readCommand
from game import Agent, Directions
from util import manhattanDistance
import numpy as np
import random
import json

In [3]:
random.seed(7)

In [4]:
def getState(state):
    currentState = (state.getPacmanPosition(),Qlearn.getNearestFoodDis(state),state.getNumFood() + len(state.getCapsules()),Qlearn.nearestGhostPos(state))
    return currentState

In [5]:
class Qlearn(Agent):
    def __init__(self, alpha,gamma,epsilon = 1,Qtable = {}):
        self.Qtable = Qtable
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.score = 0
        self.action = None
        self.currentState = None
        self.actions = []
        self.currentActions = 0
        self.rewards = []
        self.reward = 0
        self.scores = []
    
    def getQvalue(self,state,action):
        pair = str((state,action))
        return self.Qtable.get(pair,0.0)
    
    def getAction(self, state):
        actions = state.getLegalPacmanActions()
        self.score = state.getScore()
        currentState = getState(state)
        action = self.getBestAction(currentState,actions)
        state2 = state.generatePacmanSuccessor(action)
        nextState = getState(state2)
        if state2.isLose():
            reinforcement = -20000
        elif state2.isWin():
            reinforcement = 20000
        else:
            reinforcement = self.getReward(currentState,nextState,state2.getScore())
        self.learn(currentState,nextState,action,actions,reinforcement)
        self.action = action
        self.currentState = currentState
        self.currentActions += 1
        self.reward += reinforcement
        return action
        
        
    def nearestGhostPos(state):
        ghostList = state.getGhostPositions()
        nearestDistance = Qlearn.getNearGhost(state.getPacmanPosition(),ghostList)
        return nearestDistance

    def getNearGhost(pacman,ghosts):
        minDis = float("inf")
        minPos = -1
        for i in range(len(ghosts)):
            distance = manhattanDistance(pacman,ghosts[i])
            if distance < minDis:
                minDis = distance
                minPos = i
        if minPos > -1:
            return ghosts[minPos]
        else:
            return (-1,-1)

    def getNearestFoodDis(state):
        posList = Qlearn.getFoodPos(state.getFood())
        posList = posList + state.getCapsules()
        minDis = minDis = float("inf")
        #minPos = (-1,-1)
        pacPos = state.getPacmanPosition()
        for i in posList:
            distance = manhattanDistance(pacPos,i)
            if distance < minDis:
                minDis = distance
                #minPos = i
        return minDis


    def getFoodPos(grid):
        posList = []
        gridList = grid.asList()
        for i in range(len(gridList)):
            for j in range(len(gridList[i])):
                if gridList[i][j]:
                    posList.append((i,j))
        return posList

    def getBestAction(self,currentState,actions):
        if random.random() < self.epsilon:
            return random.choice(actions)
        maxValue = float("-inf")
        maxAction = []
        for i in actions:
            value = self.getQvalue(currentState,i)
            if value > maxValue:
                maxValue = value
                maxAction = [i]
            if value == maxValue:
                maxAction.append(i)
        #print(maxValue)
        #print(maxAction)
        return random.choice(maxAction)

    def learn(self,currentState,nextState,action,actions,reinforcement):
        nextActions = [self.getQvalue(nextState,a) for a in actions]
        if len(nextActions) > 0:
            newQ = max(nextActions)
        else:
            newQ = 0
        currentQ = self.getQvalue(currentState,action)
        self.Qtable[str((currentState,action))] = currentQ + self.alpha * (reinforcement + (self.gamma*newQ) - currentQ)

    def setEpsilon(self,epsilon):
        self.epsilon = epsilon

    def serialize(self,path):
        try:
            f = open(path,"w")
            json.dump(self.Qtable,f)
            f.close()
        except:
            raise Exception("You've got an error")

    def desserialize(self,path):
        try:
            f = open(path,"r")
            self.Qtable = json.load(f)
            f.close()
        except:
            raise Exception("You've got an error")
    
    def getReward(self,state1,state2,score):
        pacGhostDistance1 = manhattanDistance(state1[0],state1[3])
        pacGhostDistance2 = manhattanDistance(state2[0],state2[3])
        #fDis1 = manhattanDistance(state1[0],state1[1])
        #fDis2 = manhattanDistance(state2[0],state2[1])
        eatenFood = state2[2] - state1[2]
        ghostDistance = pacGhostDistance2 - pacGhostDistance1
        foodDistance = state2[1] - state1[1]
        if ghostDistance > 0:
            ghostReward = ghostDistance*200
        else:
            ghostReward = 200*ghostDistance
        if foodDistance < 0:
            foodReward = -500 * foodDistance
        else:
            foodReward = 0
        if eatenFood > 0:
            eatReward = 500*eatenFood
        else:
            eatReward = 0
        scoreDifference = score - self.score
        if scoreDifference > 0:
            scoreReward = 3*scoreDifference
        else:
            scoreReward = 5*scoreDifference
        return ghostReward + foodReward + eatReward + scoreReward
    
    def win(self,terminal):
        self.saveResult(terminal.state.getScore())
        self.learn(self.currentState,getState(terminal.state),self.action,terminal.state.getLegalPacmanActions(),20000)
    
    def lose(self,terminal):
        self.saveResult(terminal.state.getScore())
        self.learn(self.currentState,getState(terminal.state),self.action,terminal.state.getLegalPacmanActions(),-20000)
    
    def saveResult(self, score):
        self.actions.append(self.currentActions)
        self.scores.append(score)
        self.rewards.append(self.reward)
        self.currentActions = 0
        self.reward = 0
        self.score = 0
            

Best:  
GhostReward = 200/200  
FoodReward = -500/0  
EatReward = 500/0  
ScoreReward = 3/5  
Alpha = 0.85  
Gamma = 0.3  

First Win: 3142  
Total win:   
Average: -344.55

In [6]:
def showResults(agente,k):
    for i in range(0,k-1,1000):
        try:    
            print("Episode "+str(i+1)+": Number of actions = "+str(agente.actions[i])+"; Total score = "+str(agente.scores[i])+"; Total Reward = "+str(agente.rewards[i]))
        except(IndexError):
            print("Index out of range")
            break

In [7]:
args = readCommand(["-q", "--pacman", "LeftTurnAgent","--layout","smallClassic"])

['-q', '--pacman', 'LeftTurnAgent', '--layout', 'smallClassic']


In [8]:
a = []

In [9]:
agente = Qlearn(0.85,0.3,epsilon = 1)
args["pacman"] = agente

In [10]:
victorys = []
k = 40000
summ = 0
for i in range(k):
    agente.setEpsilon(1/(i+1))
    a.append(runGames(**args))
    if a[i][0].state.isLose():
        agente.lose(a[i][0])
    elif a[i][0].state.isWin():
        agente.win(a[i][0])
        victorys.append(a[i])
        print(i)
    summ += a[i][0].state.getScore()

Pacman emerges victorious! Score: 759
3142
Pacman emerges victorious! Score: 736
9349
Pacman emerges victorious! Score: 886
12041
Pacman emerges victorious! Score: 819
12103
Pacman emerges victorious! Score: 671
12667
Pacman emerges victorious! Score: 1026
12765
Pacman emerges victorious! Score: 894
13771
Pacman emerges victorious! Score: 569
14012
Pacman emerges victorious! Score: 625
14239
Pacman emerges victorious! Score: 631
14494
Pacman emerges victorious! Score: 638
15167
Pacman emerges victorious! Score: 683
16695
Pacman emerges victorious! Score: 660
17021
Pacman emerges victorious! Score: 815
17075
Pacman emerges victorious! Score: 868
17297
Pacman emerges victorious! Score: 395
17987
Pacman emerges victorious! Score: 859
18808
Pacman emerges victorious! Score: 640
20034
Pacman emerges victorious! Score: 863
20073
Pacman emerges victorious! Score: 711
20470
Pacman emerges victorious! Score: 516
20769
Pacman emerges victorious! Score: 834
21296
Pacman emerges victorious! Score:

In [11]:
showResults(agente,k)

Episode 1: Number of actions = 14; Total score = -454.0; Total Reward = -19073.0
Episode 1001: Number of actions = 142; Total score = -62.0; Total Reward = 22808.0
Episode 2001: Number of actions = 195; Total score = -385.0; Total Reward = 22817.0
Episode 3001: Number of actions = 568; Total score = -608.0; Total Reward = 35867.0
Episode 4001: Number of actions = 167; Total score = -407.0; Total Reward = 29797.0
Episode 5001: Number of actions = 352; Total score = -392.0; Total Reward = 48812.0
Episode 6001: Number of actions = 97; Total score = -207.0; Total Reward = 16425.0
Episode 7001: Number of actions = 113; Total score = -293.0; Total Reward = 18059.0
Episode 8001: Number of actions = 166; Total score = -256.0; Total Reward = 24944.0
Episode 9001: Number of actions = 31; Total score = -421.0; Total Reward = 4497.0
Episode 10001: Number of actions = 125; Total score = -205.0; Total Reward = 1592.0
Episode 11001: Number of actions = 154; Total score = -444.0; Total Reward = 24902.

In [12]:
agente.serialize("../../notebooks/results/small/a085-g03-d100721.json")

In [13]:
print(len(victorys))
print(summ/k)

74
-344.1665


In [25]:
argsM = readCommand(["-q", "--pacman", "LeftTurnAgent","--layout","mediumClassic"])

['-q', '--pacman', 'LeftTurnAgent', '--layout', 'mediumClassic']


In [26]:
aM = []

In [27]:
agenteM = Qlearn(0.85,0.3,epsilon = 1)
argsM["pacman"] = agenteM

In [ ]:
victorysM = []
kM = 80000
summM = 0
for i in range(kM):
    agenteM.setEpsilon(1/(i+1))
    aM.append(runGames(**argsM))
    if aM[i][0].state.isLose():
        agenteM.lose(aM[i][0])
    elif aM[i][0].state.isWin():
        agenteM.win(aM[i][0])
        victorysM.append(aM[i])
        print(i)
    summM += aM[i][0].state.getScore()

In [ ]:
agenteM.serialize("../../notebooks/results/medium/a085-g03-d090721.json")

In [ ]:
print(len(victorysM))
print(summM/kO)

In [ ]:
argsO = readCommand(["-q", "--pacman", "LeftTurnAgent","--layout","originalClassic"])

In [ ]:
aO = []

In [ ]:
agenteO = Qlearn(0.85,0.3,epsilon = 1)
argsO["pacman"] = agenteO

In [ ]:
victorysO = []
kO = 120000
summO = 0
for i in range(kO):
    agenteO.setEpsilon(1/(i+1))
    aO.append(runGames(**argsO))
    if aO[i][0].state.isLose():
        agenteO.lose(aO[i][0])
    elif aO[i][0].state.isWin():
        agenteO.win(aO[i][0])
        victorysO.append(aO[i])
        print(i)
    summ0 += aO[i][0].state.getScore()

In [ ]:
agenteO.serialize("../../notebooks/results/original/a085-g03-d090721.json")

In [ ]:
print(len(victorysO))
print(summO/kO)

In [31]:
class smartPacman(Agent):
    def __init__(self, path):
        self.desserialize(path)
        self.score = 0
        self.action = 0
    
    def getQvalue(self,state,action):
        pair = str((state,action))
        return self.Qtable.get(pair,0.0)
    
    def getAction(self, state):
        actions = state.getLegalPacmanActions()
        currentState = getState(state)
        action = self.getBestAction(currentState,actions)
        return action
    
    def getBestAction(self,currentState,actions):
        maxValue = float("-inf")
        maxAction = []
        for i in actions:
            value = self.getQvalue(currentState,i)
            if value > maxValue:
                maxValue = value
                maxAction = [i]
            if value == maxValue:
                maxAction.append(i)
        #print(maxValue)
        #print(maxAction)
        self.action += 1
        return random.choice(maxAction)
    
    def desserialize(self,path):
        try:
            f = open(path,"r")
            self.Qtable = json.load(f)
            f.close()
        except:
            raise Exception("You've got an error")
    def finish(self,state):
        self.score = state.getScore()
        self.action = 0


In [41]:
#agenteEsperto = smartPacman("../../notebooks/results/small/a085-g03-d090721-2.json")
agenteEsperto = smartPacman("../../notebooks/results/small/a085-g03-d090721-2.json")
argsV = readCommand(["--pacman", "LeftTurnAgent","--layout","smallClassic"])
argsV["pacman"] = agenteEsperto

['--pacman', 'LeftTurnAgent', '--layout', 'smallClassic']


In [44]:
a = runGames(**argsV)
print(a[0].state.isWin())
print(a[0].state.getScore())

False
-468.0
